# Valuing European Option Using the Heston Model

Heston 用於衍生性商品定價，特別是選擇權，他與 BSM 的主要區別是此模型設定【隨機波動率】，在下面的例子將對一個具有特定條件的歐式認購選擇權，進行兩個模型的定價。

In [1]:
import QuantLib as ql

設定選擇權的條件

In [47]:
# Option Data
maturity_date = ql.Date(15, 1, 2025)
spot_price = 127.62
strike_price = 130
volatility = 0.20
dividend_rate = 0.0163
option_type = ql.Option.Call

risk_free_rate = 0.05
day_count = ql.Actual365Fixed()
calendar = ql.UnitedStates(ql.UnitedStates.Settlement)

calculation_date = ql.Date(8, 5, 2023)
ql.Settings.instance().evaluationDate = calculation_date

建構歐式選擇權

In [48]:
# construct the European Option
payoff = ql.PlainVanillaPayoff(option_type, strike_price)
exercise = ql.EuropeanExercise(maturity_date)
european_option = ql.VanillaOption(payoff, exercise)

使用 Heston

In [49]:
# Heston model parameters
v0 = volitility**2
kappa = 0.1
theta = v0
sigma = 0.1
rho = -0.75

# construct the Heston process
heston_process = ql.HestonProcess(ql.YieldTermStructureHandle(ql.FlatForward(calculation_date, risk_free_rate, day_count)),
                                  ql.YieldTermStructureHandle(ql.FlatForward(calculation_date, dividend_rate, day_count)),
                                  ql.QuoteHandle(ql.SimpleQuote(spot_price)),
                                  v0, kappa, theta, sigma, rho
                                  )

# price the option using the Heston model 
engine = ql.AnalyticHestonEngine(ql.HestonModel(heston_process), 0.01, 1000)
european_option.setPricingEngine(engine)
h_price = european_option.NPV()
print("The Heston model price is ", h_price)

The Heston model price is  14.954467680045894


使用 Black-Scholes-Merton(BSM)

In [50]:
spot_handle = ql.QuoteHandle(ql.SimpleQuote(spot_price))
risk_free_rate_ts = ql.YieldTermStructureHandle(ql.FlatForward(calculation_date, risk_free_rate, day_count))
dividend_yield_ts = ql.YieldTermStructureHandle(ql.FlatForward(calculation_date, dividend_rate, day_count))
flat_vol_ts = ql.BlackVolTermStructureHandle(ql.BlackConstantVol(calculation_date, calendar, volatility, day_count))

bsm_process = ql.BlackScholesMertonProcess(spot_handle,
                                            dividend_yield_ts,
                                            risk_free_rate_ts,
                                            flat_vol_ts)

european_option.setPricingEngine(ql.AnalyticEuropeanEngine(bsm_process))
bs_price = european_option.NPV()
print("The BSM model price is ", bs_price)

The BSM model price is  15.096346835971012


### Conclusion

兩個價格不同的主因是對於【波動率】的假設不同

1. BSM，假設資產價格的波動率恆定
2. Heston，則允許波動率為一個隨機過程